In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

: 

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
NID_dataset_json={'Name':'Network Intrusion Detection',
                  'path_train_csv':'/kaggle/input/network-intrusion-detection/Train_data.csv',
                  'path_test_csv': '/kaggle/input/network-intrusion-detection/Test_data.csv',
                  'target_field':'class'
                 }

NSLKDD_dataset_json={'title':'NSL KDD',
                     'path_train_csv':'/conndataset/Conn.csv',
                     'path_test_csv': '/kaggle/input/nsl-dataset/test500.csv',
                     'target_field':'class'
                    }
NIDS_dataset_json={'title':'NIDS V2',
                     'path_train_csv':'/kaggle/input/nfuqnidsv2-network-intrusion-detection-dataset/NF-UQ-NIDS-v2.csv',
                     'target_field':'class'
                    }

hyperparameter={'epoch':100,
                'batch-size':32,
                'loss-function':'triplet_loss'
               }


os.mkdir('model') if not os.path.isdir('model') else None
os.mkdir('model/checkpoint') if not os.path.isdir('model/checkpoint') else None

dataset_json=NSLKDD_dataset_json

: 

This code defines a load_and_preprocess_dataset function that takes as input the path to a CSV file containing the dataset and an optional n_components parameter specifying the number of features to select. The function loads the dataset, preprocesses it by scaling the features and encoding any categorical variables, applies PCA to reduce its dimensionality, and visualizes the resulting PCA using a scatter plot. The function then selects the top n_components features based on their contribution to the first principal component and returns these selected features along with the labels.

In [8]:
import pandas as pd

# Load the data from the CSV or Excel file
data = pd.read_csv(dataset_json['path_train_csv']) # or pd.read_excel('data.xlsx')

# Calculate the number of rows to select
n_rows = int(len(data) / 2)

# Select the first half of the data
data_first_half = data.iloc[:n_rows, :]

# Select the second half of the data
data_second_half = data.iloc[n_rows:, :]

# Preprocess the selected data
# ...


In [9]:
from sklearn.preprocessing import LabelEncoder

def load_and_preprocess_dataset(path_csv, n_components=14):
    df = data_first_half#pd.read_csv(path_csv)
    
    # Drop columns with zero variance
    zero_variance_columns = ['missed_bytes']
    #zero_variance_columns = ['is_host_login"', 'num_outbound_cmds', 'num_shells', 'root_shell', 'urgent', 'land']
    df.drop(columns=zero_variance_columns, inplace=True)
    
    # Split into features and labels
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    # Encode categorical features using one-hot encoding
    categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
    X = pd.get_dummies(data=X, columns=categorical_columns)
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Create an instance of the PCA class
    pca = PCA(n_components=n_components)

    # Fit the PCA model to the data
    X_pca = pca.fit_transform(X_scaled)

    # Encode labels using label encoding
    label_encoder = LabelEncoder().fit(y)
    y_encoded = label_encoder.transform(y)
    
    # Visualize the resulting PCA
    plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y_encoded)
    plt.xlabel('First Principal Component')
    plt.ylabel('Second Principal Component')
    plt.show()

    return X_pca, y_encoded


In [ ]:
import tensorflow as tf


def preprocess_data(dataset_path):
    # Load the data from the CSV file
    dataset = tf.data.experimental.CsvDataset(
        dataset_path,
        record_defaults, # define the data types for each column
        header=True,
        field_delim=','
    )

    # Apply preprocessing operations to the data
    dataset = dataset.map(preprocess_function) # define a function to preprocess the data

    # Batch the data
    dataset = dataset.batch(batch_size)

    return dataset


In [ ]:
# Load and preprocess dataset
X,y=load_and_preprocess_dataset(dataset_json['path_train_csv'])


In [ ]:
X,y=load_and_preprocess_dataset(data_first_half)

/tmp/ipykernel_32/1244496347.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=zero_variance_columns, inplace=True)


In [ ]:


def create_triplet_samples(x_train, y_train, n_features, n_batch):
    x_anchors = np.zeros((n_batch, n_features))
    x_positives = np.zeros((n_batch, n_features))
    x_negatives = np.zeros((n_batch, n_features))
    
    for i in range(n_batch):
        # Select a random sample from x_train as the anchor
        anchor_idx = np.random.randint(0, len(x_train))
        x_anchor = x_train[anchor_idx]
        y_anchor = y_train[anchor_idx]
        
        # Find the indices of samples with the same label as the anchor
        pos_indices = np.where(y_train == y_anchor)[0]
        # Select a random positive sample
        pos_idx = np.random.choice(pos_indices)
        x_positive = x_train[pos_idx]
        
        # Find the indices of samples with a different label than the anchor
        neg_indices = np.where(y_train != y_anchor)[0]
        # Select a random negative sample
        neg_idx = np.random.choice(neg_indices)
        x_negative = x_train[neg_idx]
        
        # Add the samples to the batch
        x_anchors[i] = x_anchor
        x_positives[i] = x_positive
        x_negatives[i] = x_negative
    
    return [x_anchors, x_positives, x_negatives]

def triplet_loss(alpha):
    def loss(y_true, y_pred):
        anchor, positive, negative = tf.split(y_pred, 3, axis=1)
        positive_dist = tf.reduce_mean(tf.square(anchor - positive), axis=1)
        negative_dist = tf.reduce_mean(tf.square(anchor - negative), axis=1)
        return tf.maximum(positive_dist - negative_dist + alpha, 0.)
    return loss

def build_siamese_model(num_of_features, emb_size):
    embedding_model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(num_of_features,)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1024, activation='relu'),
        #tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(2048, activation='relu'),
        #tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4096, activation='relu'),
        #tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(emb_size, activation='sigmoid')
    ])



    input_anchor = tf.keras.layers.Input(shape=(num_of_features,))
    input_positive = tf.keras.layers.Input(shape=(num_of_features,))
    input_negative = tf.keras.layers.Input(shape=(num_of_features,))
    
    embedding_anchor = embedding_model(input_anchor)
    embedding_positive = embedding_model(input_positive)
    embedding_negative = embedding_model(input_negative)

    output = tf.keras.layers.concatenate([embedding_anchor, embedding_positive, embedding_negative], axis=1)

    siamese_model = tf.keras.models.Model([input_anchor, input_positive, input_negative], output)
    
    return siamese_model

def train_siamese_model(model, X_train, y_train, batch_size):
    data_gen = data_generator(batch_size,X_train,y_train)
    
    #optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss=triplet_loss(alpha=0.01), optimizer='adam',run_eagerly=True)
    history=model.fit(data_gen,
                      epochs=120,
                      steps_per_epoch=int(len(X_train)/batch_size),
                      verbose=2)
    
    loss = history.history['loss']

    # Plot the training loss
    plt.plot(loss)
    plt.title('Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()
    
   

def data_generator(batch_size,x_dataset,y_dataset):
    while True:
        x=create_triplet_samples(x_dataset,y_dataset,num_of_features,batch_size)
        y=np.zeros((batch_size,num_of_features*3))
        
        yield x,y

# Load and preprocess dataset
#X,y=load_and_preprocess_dataset(dataset_json['path_train_csv'])

# Split into training and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

# Reshape and normalize data
X_train=np.reshape(X_train,(X_train.shape[0],X_train.shape[1]))/255.
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1]))/255.

# Build and train Siamese model
num_of_features=X_train.shape[1]
emb_size=64
siamese_model=build_siamese_model(num_of_features,emb_size)
train_siamese_model(siamese_model,X_train,y_train,batch_size=64)


In [ ]:
def create_test_triplets(x_test, y_test, n_features, n_batch):
    while True:
        x_anchors = np.zeros((n_batch, n_features))
        x_positives = np.zeros((n_batch, n_features))
        x_negatives = np.zeros((n_batch, n_features))

        for i in range(n_batch):
            # Select a random sample from x_test as the anchor
            anchor_idx = np.random.randint(0, len(x_test))
            x_anchor = x_test[anchor_idx]
            y_anchor = y_test[anchor_idx]

            # Find the indices of samples with the same label as the anchor
            pos_indices = np.where(y_test == y_anchor)[0]
            # Select a random positive sample
            pos_idx = np.random.choice(pos_indices)
            x_positive = x_test[pos_idx]

            # Find the indices of samples with a different label than the anchor
            neg_indices = np.where(y_test != y_anchor)[0]
            # Select a random negative sample
            neg_idx = np.random.choice(neg_indices)
            x_negative = x_test[neg_idx]

            # Add the samples to the batch
            x_anchors[i] = x_anchor
            x_positives[i] = x_positive
            x_negatives[i] = x_negative

        yield [x_anchors, x_positives, x_negatives]

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def evaluate_siamese_model(model, X_test, y_test, batch_size):
    # Create a test data generator
    test_data_gen = create_test_triplets(X_test, y_test, num_of_features, batch_size)
    
    # Compute the embeddings for the test data
    n_batches = int(np.ceil(len(X_test) / batch_size))
    embeddings = []
    for i in range(n_batches):
        x_test_batch = next(test_data_gen)
        embeddings_batch = model.predict(x_test_batch)
        embeddings.append(embeddings_batch)
    embeddings = np.concatenate(embeddings)
    
    # Compute the pairwise distances between all embeddings
    n_embeddings = len(embeddings)
    dists = np.zeros((n_embeddings, n_embeddings))
    for i in range(n_embeddings):
        for j in range(n_embeddings):
            dists[i, j] = np.linalg.norm(embeddings[i] - embeddings[j])
    
    # Compute the predictions and true labels for all pairs of samples
    y_true = []
    y_pred = []
    for i in range(len(y_test)):
        for j in range(i+1, len(y_test)):
            if y_test[i] == y_test[j]:
                # Samples i and j have the same label
                y_true.append(1)
                y_pred.append(dists[i, j] < 0.5)
            else:
                # Samples i and j have different labels
                y_true.append(0)
                y_pred.append(dists[i, j] >= 0.5)
                
    
    # Compute the accuracy, precision, and F1 score
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot the confusion matrix
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    
    return accuracy, precision, f1

# Evaluate the Siamese model on the test data
accuracy, precision, f1 = evaluate_siamese_model(siamese_model, X_test, y_test,batch_size=64)
print(f'Test accuracy: {accuracy:.2f}')
print(f'Test precision: {precision:.2f}')
print(f'Test F1 score: {f1:.2f}')


# for classification (optional)

In [ ]:
def build_embedding_model(num_of_features, emb_size):
    embedding_model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(num_of_features,)),
        tf.keras.layers.Dense(emb_size, activation='sigmoid')
    ])
    
    return embedding_model

def evaluate_siamese_model(model, embedding_model, X_test, y_test):
    # Compute the embeddings for the test set
    test_embeddings = embedding_model.predict(X_test)
    
    # Compute the pairwise distances between all embeddings
    dist_matrix = np.zeros((len(test_embeddings), len(test_embeddings)))
    for i in range(len(test_embeddings)):
        for j in range(len(test_embeddings)):
            dist_matrix[i,j] = np.linalg.norm(test_embeddings[i] - test_embeddings[j])
    
    # Compute the average distance between embeddings of the same class
    same_class_dist = []
    for i in range(len(test_embeddings)):
        same_class_indices = np.where(y_test == y_test[i])[0]
        same_class_dist.append(np.mean(dist_matrix[i,same_class_indices]))
    same_class_dist = np.mean(same_class_dist)
    
    # Compute the average distance between embeddings of different classes
    diff_class_dist = []
    for i in range(len(test_embeddings)):
        diff_class_indices = np.where(y_test != y_test[i])[0]
        diff_class_dist.append(np.mean(dist_matrix[i,diff_class_indices]))
    diff_class_dist = np.mean(diff_class_dist)
    
    print(f'Average distance between embeddings of the same class: {same_class_dist:.4f}')
    print(f'Average distance between embeddings of different classes: {diff_class_dist:.4f}')

# Build embedding model
embedding_model = build_embedding_model(num_of_features, emb_size)

# Set weights of embedding model to be the same as the Siamese model
embedding_model.set_weights(siamese_model.layers[3].get_weights())

# Evaluate Siamese model on test set
evaluate_siamese_model(siamese_model, embedding_model, X_test, y_test)


# Visualization using t-SNE

However, it’s important to note that this Siamese network model is an unsupervised learning model that   embed data points into a lower-dimensional space such that similar data points are close together and dissimilar data points are far apart. As such, traditional classification metrics like accuracy, precision, recall, and F1 score are not applicable to this type of model.

Instead, we use other methods to evaluate the performance of the model, such as visualizing the embeddings using dimensionality reduction techniques like t-SNE or UMAP. Here we created a 2D visualization of the embeddings using t-SNE:

t-SNE (t-Distributed Stochastic Neighbor Embedding) is a nonlinear dimensionality reduction technique that is particularly effective at preserving the local structure of the data. It works by minimizing the divergence between two distributions: a distribution that measures pairwise similarities of the input objects and a distribution that measures pairwise similarities of the corresponding low-dimensional points in the embedding1.

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_embeddings(embedding_model, X_test, y_test):
    # Compute the embeddings for the test set
    test_embeddings = embedding_model.predict(X_test)
    
    # Reduce the dimensionality of the embeddings to 2D using t-SNE
    tsne = TSNE(n_components=2,perplexity=10)
    tsne_embeddings = tsne.fit_transform(test_embeddings)
    
    # Create a scatter plot of the 2D embeddings
    plt.scatter(tsne_embeddings[:,0], tsne_embeddings[:,1], c=y_test, cmap='tab10')
    plt.colorbar()
    plt.show()

# Visualize embeddings of test set
visualize_embeddings(embedding_model, X_test, y_test)


This code computes the embeddings for the test set using the predict method of the embedding_model, then reduces the dimensionality of the embeddings to 2D using t-SNE. Finally, it creates a scatter plot of the 2D embeddings, where each point is colored according to its class label.

This scatter plot shows the 2D visualization of the embeddings of our data points using the t-SNE technique. The plot has four distinct clusters of data points, each represented by a different color. The color scale on the right side of the plot ranges from 1 to 10, indicating that there are 10 different classes in our data.

The t-SNE technique is designed to preserve the local structure of the data, so data points that are close together in the high-dimensional space are likely to be close together in the 2D visualization as well. From the plot, we can see that the four clusters are well-separated from each other, indicating that the Siamese network model is able to learn a good representation of the data that separates the different classes.

# Using PCA visulaization
This code computes the embeddings for the test set using the predict method of the embedding_model, then reduces the dimensionality of the embeddings to 2D using PCA. Finally, it creates a scatter plot of the 2D embeddings, where each point is colored according to its class label.

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def visualize_embeddings_pca(embedding_model, X_test, y_test):
    # Compute the embeddings for the test set
    test_embeddings = embedding_model.predict(X_test)
    
    # Reduce the dimensionality of the embeddings to 2D using PCA
    pca = PCA(n_components=2)
    pca_embeddings = pca.fit_transform(test_embeddings)
    
    # Create a scatter plot of the 2D embeddings
    plt.scatter(pca_embeddings[:,0], pca_embeddings[:,1], c=y_test, cmap='tab10')
    plt.colorbar()
    plt.show()

# Visualize embeddings of test set using PCA
visualize_embeddings_pca(embedding_model, X_test, y_test)
